Party-Adaptive XGBoost

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from ddsketch.ddsketch import DDSketch

In [51]:
file = 'RELATHE.csv'
df_main = pd.read_csv(file)
df_main = df_main.sample(frac = 1)
df_main.shape

(1427, 4323)

In [52]:
df_main.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V4314,V4315,V4316,V4317,V4318,V4319,V4320,V4321,V4322,TARGET
207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# DATA DIVISION

df_1 = df_main.iloc[:300, :]
df_2 = df_main.iloc[300:600, :]
df_3 = df_main.iloc[600:900, :]
df_4 = df_main.iloc[900:1200, :]
df_5 = df_main.iloc[1200:, :]

df_list = [df_1, df_2, df_3, df_4, df_5]

In [54]:
global_model = pickle.load(open("xgb_clf.dat", "rb"))

In [59]:
class Client():
    def __init__(self, cli, df, model, eps):
        self.cli = cli
        self.df = df
        self.model = model
        self.eps = eps
        self.X = df.drop('TARGET', axis = 1)
        self.y = df['TARGET']
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size = 0.2, random_state = 0)
        self.dtrain = xgb.DMatrix(X_train, label=y_train)
        self.dtest = xgb.DMatrix(X_test, label=y_test)
        self.eval_set = [(X_test, y_test)]
    def train():
        return
    def update_weights():
        return
    def compute_sketch(self):
        #returns a list of sketches for each variable
        h, w = self.df.shape
        local_sketch = []
        for i in range(w):
            sketch = DDSketch()
            for val in range(h):
                sketch.add(self.df.iloc[val, i])
            local_sketch.append(sketch.__repr__)
        return local_sketch
    def generate_predictions(self):
        model = self.model
        preds = model.predict(self.dtest)
        preds = np.asarray([int(pred>=0.5) for pred in preds])
        return preds
    def compute_gh(self):
        y = self.dtest.get_label()
        predt = self.model.predict(self.dtest)
        grad = (np.log1p(predt) - np.log1p(y)) / (predt + 1)
        hess = ((-np.log1p(predt) + np.log1p(y) + 1) / np.power(predt + 1, 2))
        return grad, hess 
    def generate_weights(self):
        df = self.df
        model = self.model
        X = df.drop('TARGET', axis = 1)
        y = df['TARGET']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        eval_set = [(X_test, y_test)]
        model.fit(X_train, 
                y_train, 
                early_stopping_rounds=15,                  
                eval_set=eval_set,
                verbose = False)
        results = model.evals_result()
        return model.get_booster().trees_to_dataframe(), sum(results['validation_0']['logloss'])/len(results['validation_0']['logloss'])
    def inference(self, model):    #return the acc, loss of given model on local dataset
        df = self.df
        self.model = model
        X = df.drop('TARGET', axis = 1)
        y = df['TARGET']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        y_pred = model.predict(X_test)
        return accuracy_score(y_test, y_pred), 

In [60]:
cl1 = Client(1, df_1, global_model, 0)
cl2 = Client(2, df_2, global_model, 0)
cl3 = Client(3, df_3, global_model, 0)
cl4 = Client(4, df_4, global_model, 0)
cl5 = Client(5, df_5, global_model, 0)

cli_list = [cl1, cl2, cl3, cl4, cl5]

In [65]:
cl1.__dict__

{'cli': 1,
 'df':       V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V4314  V4315  V4316  \
 207    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 1351   0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 721    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 272    1   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 340    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 ...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...    ...    ...    ...   
 433    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 398    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 611    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 392    0   0   0   0   0   0   0   0   1    0  ...      0      0      0   
 208    0   0   0   0   0   0   0   0   0    0  ...      0      0      0   
 
       V4317  V4318  V4319  V4320  V4321  V4322  TARGET  
 207       

In [61]:
epsa = 0.1    #Global Error Tolerance Budget OR number of maximum bins of histogram
eps_list = []

def compute_local_epsilon(epsa):
    s = len(df_main)
    for cli in range(5):
        cli_list[cli].eps = epsa*(len(df_list[cli])/s)
        eps_list.append(cli_list[cli].eps)
    return eps_list

In [62]:
df_hist = []
for cli in range(5):
    df_hist.append(cli_list[cli].compute_sketch())

KeyboardInterrupt: 

In [ ]:
def merge_hist():
    return

#### REMAINING: 
 MERGE HISTOGRAM <br>
 GROW TREE

# DRIVER CODE

In [ ]:
global_training_rounds = 4
for t in range(global_training_rounds):
    df_hist_agg = []
    grad_agg = []
    hess_agg = []
    for i in range(5):
        cli_list[i].model = global_model
        y_pred = cli_list[i].generate_predictions()
        #COMPUTE GRADIENT AND HESSIAN
        grad, hess = cli_list[i].compute_gh()
        hist_agg.append(df_hist[i])
        grad_agg.append(grad_agg, grad)
        hess_agg.append(hess_agg, hess)
    epsa = min(eps_list)    #min for classification objective
        
    '''hist_agg, gm_agg, hm_agg = merge_hist(hist_agg, grad_agg, hess_agg, epsa)
    global_model = grow_tree(hist_agg, gm_agg, hm_agg)'''

In [ ]:
# NEW APPROACH: Share data directly between parties

In [ ]:
# POSSIBLE LOOPHOLE: predictions and grad, hess computed only for test set of respective data